In [1]:
import os

os.chdir('../')
os.getcwd()

'c:\\Users\\91629\\Desktop\\projects\\SmartPoultryMgt'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [6]:
import os
import gdown
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Convert Google Drive URL to direct download URL
            file_id = self.config.source_URL.split('/')[-2]  # Extract file ID
            download_url = f"https://drive.google.com/uc?id={file_id}"

            # Download file using gdown
            gdown.download(download_url, str(self.config.local_data_file), quiet=False)
            
            logger.info(f"File downloaded successfully: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """ Extracts the ZIP file into the data directory """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed at {unzip_path}")
        except zipfile.BadZipFile:
            logger.error("Downloaded file is not a valid ZIP. Check the source file.")
            raise


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-16 15:00:44,722: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 15:00:44,724: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 15:00:44,726: INFO: common: created directory at: artifacts]
[2025-03-16 15:00:44,730: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?id=12Nuhzc60vG_qeoRS-YK2G0ba_bmvEnde
From (redirected): https://drive.google.com/uc?id=12Nuhzc60vG_qeoRS-YK2G0ba_bmvEnde&confirm=t&uuid=80049319-acb2-4b10-bf23-ad21687fcd6b
To: c:\Users\91629\Desktop\projects\SmartPoultryMgt\artifacts\data_ingestion\data.zip
100%|██████████| 714M/714M [03:18<00:00, 3.59MB/s] 

[2025-03-16 15:04:09,151: INFO: 1894831921: File downloaded successfully: artifacts/data_ingestion/data.zip]


[2025-03-16 15:04:36,851: INFO: 1894831921: Extraction completed at artifacts/data_ingestion]
